In [1]:
# Paso 1: Conectar con Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Paso 2: Instalar y actualizar las librerías (Método Minimalista y Correcto)
# Esto respeta el entorno de Colab y solo actualiza lo que necesitamos.
print("\nInstalando y actualizando librerías...")
!pip install --upgrade -q mne pytorch-lightning timm
print("✅ Librerías listas.")

Mounted at /content/drive

Instalando y actualizando librerías...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
import pytorch_lightning as pl
from torchmetrics.classification import MulticlassAccuracy, MulticlassF1Score, MulticlassCohenKappa
import numpy as np

# ==============================================================================
# DEFINICIÓN DE LA ARQUITECTURA DEL MODELO
# ==============================================================================
def get_convnext_model(num_classes=5, pretrained=True):
    model = timm.create_model('convnextv2_tiny.fcmae_ft_in22k_in1k', pretrained=pretrained)
    original_conv = model.stem[0]
    new_first_conv = nn.Conv2d(1, original_conv.out_channels,
                               kernel_size=original_conv.kernel_size,
                               stride=original_conv.stride,
                               padding=original_conv.padding,
                               bias=(original_conv.bias is not None))
    with torch.no_grad():
        new_first_conv.weight[:, :] = original_conv.weight.clone().mean(dim=1, keepdim=True)
    model.stem[0] = new_first_conv
    num_ftrs = model.head.fc.in_features
    model.head.fc = nn.Linear(num_ftrs, num_classes)
    return model

# ==============================================================================
# DEFINICIÓN DEL CLASIFICADOR (LÓGICA DE ENTRENAMIENTO)
# ==============================================================================
class SleepStageClassifier(pl.LightningModule):
    def __init__(self, model_name='convnextv2_tiny', num_classes=5, learning_rate=1e-4, class_weights=None):
        super().__init__()
        self.save_hyperparameters()
        self.model = get_convnext_model(num_classes=num_classes, pretrained=True)

        # Métricas para cada fase (versión corregida y compatible)
        self.train_metrics = nn.ModuleDict({'acc': MulticlassAccuracy(num_classes=num_classes), 'f1': MulticlassF1Score(num_classes=num_classes)})
        self.val_metrics = nn.ModuleDict({'acc': MulticlassAccuracy(num_classes=num_classes), 'f1': MulticlassF1Score(num_classes=num_classes)})
        self.test_metrics = nn.ModuleDict({'acc': MulticlassAccuracy(num_classes=num_classes), 'f1': MulticlassF1Score(num_classes=num_classes), 'kappa': MulticlassCohenKappa(num_classes=num_classes)})

        self.weights = torch.tensor(class_weights, dtype=torch.float) if class_weights is not None else None
        self.loss_fn = F.cross_entropy

    def forward(self, x):
        return self.model(x)

    def _common_step(self, batch, batch_idx):
        x, y_true = batch
        y_pred_logits = self(x)
        if self.weights is not None:
            loss = self.loss_fn(y_pred_logits, y_true, weight=self.weights.to(self.device))
        else:
            loss = self.loss_fn(y_pred_logits, y_true)
        preds = torch.argmax(y_pred_logits, dim=1)
        return loss, preds, y_true

    def training_step(self, batch, batch_idx):
        loss, preds, y_true = self._common_step(batch, batch_idx)
        metrics = {name: metric(preds, y_true) for name, metric in self.train_metrics.items()}
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log_dict({f'train_{k}': v for k, v in metrics.items()}, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, preds, y_true = self._common_step(batch, batch_idx)
        metrics = {name: metric(preds, y_true) for name, metric in self.val_metrics.items()}
        self.log('val_loss', loss, prog_bar=True)
        self.log_dict({f'val_{k}': v for k, v in metrics.items()}, on_epoch=True)

    def test_step(self, batch, batch_idx):
        loss, preds, y_true = self._common_step(batch, batch_idx)
        metrics = {name: metric(preds, y_true) for name, metric in self.test_metrics.items()}
        self.log('test_loss', loss, prog_bar=True)
        self.log_dict({f'test_{k}': v for k, v in metrics.items()}, on_epoch=True)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.hparams.learning_rate)

print("✅ Clases y funciones del modelo definidas exitosamente.")

✅ Clases y funciones del modelo definidas exitosamente.


In [8]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from pathlib import Path
import pytorch_lightning as pl
import logging
import numpy as np

# --- Optimización para GPUs Modernas ---
torch.set_float32_matmul_precision('medium')

# --- Dataset para cargar los datos de SHHS (sin cambios) ---
class SHHSDataset(Dataset):
    def __init__(self, file_paths):
        self.file_paths = file_paths
        logging.info(f"Dataset de prueba (SHHS) inicializado con {len(self.file_paths)} sujetos.")

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        df = pd.read_parquet(self.file_paths[idx])
        valid_labels = [0, 1, 2, 3, 4]
        df = df[df['label'].isin(valid_labels)]
        spectrogram_flat = df.iloc[:, :-1].values.astype(np.float32)
        labels = df['label'].values.astype(np.int64)
        spectrogram_2d = spectrogram_flat.reshape(-1, 1, 76, 60)
        return torch.from_numpy(spectrogram_2d), torch.from_numpy(labels)

# ------------------------------------------------------------------------------------
# RUTAS (Sin cambios)
# ------------------------------------------------------------------------------------
shhs_processed_dir = Path('/content/drive/MyDrive/shhs_processed/')
path_al_modelo_campeon = Path('/content/drive/MyDrive/sleep_model_checkpoints/best-model-epoch=10-val_loss=1.34.ckpt')
# ------------------------------------------------------------------------------------

# --- Preparar los datos de prueba ---
all_shhs_files = sorted(list(shhs_processed_dir.glob('**/*.parquet')))
sample_files = all_shhs_files[:200]
shhs_dataset = SHHSDataset(sample_files)

def collate_fn(batch):
    spectrograms, labels = zip(*batch)
    return torch.cat(spectrograms, 0), torch.cat(labels, 0)

# --- ¡CORRECCIÓN CLAVE AQUÍ! ---
# Cambiamos num_workers de 2 a 0 para desactivar la carga en paralelo y evitar el error.
test_loader = DataLoader(shhs_dataset, batch_size=4, shuffle=False, num_workers=0, collate_fn=collate_fn)
# --------------------------------

# --- Cargar el modelo y ejecutar la evaluación ---
try:
    if not path_al_modelo_campeon.exists():
        raise FileNotFoundError(f"¡ERROR DE RUTA! No se encontró el archivo del modelo en: {path_al_modelo_campeon}")

    model = SleepStageClassifier.load_from_checkpoint(path_al_modelo_campeon)
    trainer = pl.Trainer(accelerator='gpu', devices=1)

    logging.info("===================================================")
    logging.info("--- Iniciando la Prueba de Generalización en SHHS... ---")
    logging.info("===================================================")

    results = trainer.test(model, test_loader)

    print("\n===================================================")
    print("--- Resultados Finales de la Prueba de Generalización ---")
    print("===================================================")
    for key, value in results[0].items():
        print(f"{key.replace('test_', '').capitalize():<10}: {value:.4f}")

except Exception as e:
    logging.error(f"Ha ocurrido un error: {e}")

INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.19999998807907104    │
│          test_f1          │    0.09396857768297195    │
│        test_kappa         │            0.0            │
│         test_loss         │    1.8418959379196167     │
└───────────────────────────┴───────────────────────────┘


--- Resultados Finales de la Prueba de Generalización ---
Loss      : 1.8419
Acc       : 0.2000
F1        : 0.0940
Kappa     : 0.0000
